In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
import ast
import matplotlib.pyplot as plt
import tensorflow as tf
import json
import os
from tqdm import tqdm_notebook
import glob
import sklearn.utils
import ntpath

In [7]:
files = glob.glob('../input/train_simp/*.csv')

In [13]:
line_cnts = []
for file in files:
    head, tail = ntpath.split(file)
#     print(tail)
    tmp = pd.read_csv(file)
    print(tail, tmp.shape[0])
    line_cnts.append({'file':tail,'line':tmp.shape[0]})

airplane.csv 151623
alarm clock.csv 123399
ambulance.csv 148004
angel.csv 149736
animal migration.csv 137847
ant.csv 124612
anvil.csv 126231
apple.csv 144722
arm.csv 120951
asparagus.csv 168102
axe.csv 124122
backpack.csv 125801
banana.csv 307936
bandage.csv 147614
barn.csv 151139
baseball bat.csv 123809
baseball.csv 135375
basket.csv 118458
basketball.csv 133793
bat.csv 118114
bathtub.csv 174336
beach.csv 124938
bear.csv 134762
beard.csv 165202
bed.csv 113862
bee.csv 120890
belt.csv 191119
bench.csv 128695
bicycle.csv 126527
binoculars.csv 124190
bird.csv 133572
birthday cake.csv 144982
blackberry.csv 128153
blueberry.csv 127878
book.csv 119364
boomerang.csv 142682
house plant.csv 122996
house.csv 135420
hurricane.csv 136245
ice cream.csv 123133
jacket.csv 214124
jail.csv 120131
kangaroo.csv 174470
key.csv 160965
keyboard.csv 187766
knee.csv 267540
ladder.csv 125389
lantern.csv 149912
laptop.csv 261501
leaf.csv 125571
leg.csv 116804
light bulb.csv 120879
lighthouse.csv 160903
lightnin

In [14]:
line_cnt = pd.DataFrame(line_cnts)

In [16]:
line_cnt.to_csv('../input/line_cnts.csv', index=False)

In [40]:
NUM_EACH_CLASS = 20000
select_nrows = NUM_EACH_CLASS
skip_rows = NUM_EACH_CLASS * 8
end_rows = skip_rows + select_nrows

def calc_skip(lines):
    if lines < end_rows:
        ret = abs(end_rows - lines)
        if (lines - ret) < select_nrows:
            ret = lines - select_nrows
        return ret
    else:
        return skip_rows

line_cnt = pd.read_csv('../input/line_cnts.csv')
line_cnt['skip_rows'] = line_cnt['line'].apply(calc_skip)
line_cnt['val_skip_rows'] = line_cnt['line'].apply(lambda x : x - 100)

In [41]:
line_cnt

,file,line,skip_rows,val_skip_rows
0,airplane.csv,151623,28377,151523
1,alarm clock.csv,123399,56601,123299
2,ambulance.csv,148004,31996,147904
3,angel.csv,149736,30264,149636
4,animal migration.csv,137847,42153,137747
5,ant.csv,124612,55388,124512
6,anvil.csv,126231,53769,126131
7,apple.csv,144722,35278,144622
8,arm.csv,120951,59049,120851
9,asparagus.csv,168102,11898,168002


In [5]:
train = pd.read_csv('../input/train_simp/angel.csv', skiprows=range(1, 149736), nrows=100)

In [6]:
train.shape

(1, 6)

In [14]:
train.drawing = train.drawing.apply(json.loads)

In [15]:
train['num_strokes'] = train.drawing.apply(len)

In [17]:
train.num_strokes.max()

99

In [2]:
STEPS = 800
EPOCHS = 16
size = 64
batchsize = 680


BASE_SIZE = 256
NCSVS = 100
NCATS = 340
np.random.seed(seed=1987)
tf.set_random_seed(seed=1987)

def f2cat(filename: str) -> str:
    return filename.split('.')[0]

def list_all_categories():
    files = os.listdir(os.path.join(INPUT_DIR, 'train_simplified'))
    return sorted([f2cat(f) for f in files], key=str.lower)

In [3]:
 def draw_cv2(raw_strokes, size=256, lw=2, time_color=True):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

In [4]:
train_cvs = glob.glob('../input/train_simp/*.csv')

In [40]:
trn_cvs = []
val_cvs = []
oof_cvs = []
for path in tqdm_notebook(train_cvs):
    train0 = pd.read_csv(path)[['drawing', 'word']].sample(113613)
    train0 = sklearn.utils.shuffle(train0)
    trn_cvs.append(train0[:-9100])
    oof_cvs.append(train0[-9100:-100])
    val_cvs.append(train0[-100:])

In [41]:
trn_cvs = pd.concat(trn_cvs)
val_cvs = pd.concat(val_cvs)
oof_cvs = pd.concat(oof_cvs)

trn_cvs.reset_index(inplace=True)
val_cvs.reset_index(inplace=True)
oof_cvs.reset_index(inplace=True)

In [42]:
trn_cvs = trn_cvs[['drawing', 'word']]
val_cvs = val_cvs[['drawing', 'word']]
oof_cvs = oof_cvs[['drawing', 'word']]

In [10]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [11]:
le.fit(trn_cvs.word)

LabelEncoder()

In [43]:
oof_cvs.word = le.transform(oof_cvs.word)
trn_cvs.word = le.transform(trn_cvs.word)
val_cvs.word = le.transform(val_cvs.word)

In [44]:
trn_cvs = sklearn.utils.shuffle(trn_cvs)
oof_cvs = sklearn.utils.shuffle(oof_cvs)
val_cvs = sklearn.utils.shuffle(val_cvs)

In [45]:
trn_cvs.to_csv('../input/train_all_balance.csv', index = False)
val_cvs.to_csv('../input/valid_balance.csv', index = False)
oof_cvs.to_csv('../input/oof_balance.csv', index = False)

In [ ]:
classes = list(le.classes_)
classes = [name.replace(' ', '_') for name in classes]
np_classes = np.array( classes)
np.save('../input/classes.np', np_classes)

In [46]:
trn_cvs.word.value_counts()

339    104513
115    104513
107    104513
108    104513
109    104513
110    104513
111    104513
112    104513
113    104513
114    104513
116    104513
126    104513
117    104513
118    104513
119    104513
120    104513
121    104513
122    104513
123    104513
124    104513
106    104513
105    104513
104    104513
103    104513
86     104513
87     104513
88     104513
89     104513
90     104513
91     104513
        ...  
245    104513
246    104513
247    104513
248    104513
249    104513
250    104513
251    104513
252    104513
235    104513
233    104513
214    104513
232    104513
215    104513
216    104513
217    104513
218    104513
219    104513
220    104513
221    104513
222    104513
223    104513
224    104513
225    104513
226    104513
227    104513
228    104513
229    104513
230    104513
231    104513
0      104513
Name: word, Length: 340, dtype: int64

In [ ]:
trn_csv.head()

In [ ]:
trn_csv = sklearn.utils.shuffle(trn_csv)

In [ ]:
trn_csv.to_csv('../input/train_all.csv', index = False)

In [18]:
trn_cvs.shape

(46613579, 2)